In [1]:
import numpy as np
import pandas as pd

from util_io import (
    init, finalize, dump_conf, assigned_day_to_family_on_day, assigned_day_to_occupancy
)
from util_cost import (
    cal_total, n_people, family_id_choice_to_pref_cost, cal_total_preference, cal_total_accounting,
    nd_ndp1_to_account_penality
)
from util_cost import choices as family_pref
from util_check import deep_check

In [2]:
# constants #
N_families = 5000
N_days = 100
N_min_people = 125
N_max_people = 300
# constants #

# params #
# path_init_conf =     '../output/m08-improved-test.csv'
path_dump_improved = '../output/m13-improved-v2.csv' # lowest cost
num_cpu_cores = 4
time_limit = 12*60*60*1000  # in ms
#time_limit = 60*1000  # in ms

In [3]:
# assigned_day, family_on_day, occupancy = init(path_conf=path_init_conf)
# print('Init config:')
# try:
#     is_valid = deep_check(assigned_day, family_on_day, occupancy)
# except:
#     is_valid = False
# print('Valid solution:', is_valid)
# print('Total score:    ', cal_total(assigned_day, occupancy))
# print('Preference cost:', cal_total_preference(assigned_day))
# print('Accounting cost:', cal_total_accounting(occupancy))

Read initial configs...
Read config completed.
Init config:
Total score:     73917.13994586898
Preference cost: 68929
Accounting cost: 4988.139945868992


In [4]:
len(assigned_day)

5000

## Setup

In [5]:
families = range(N_families)
days = range(1, N_days + 1)

In [6]:
allowed_occupancy = range(N_min_people, N_max_people+1)

In [7]:
# Possible choice for the family
# last choice is any day that is not on the family's preferred days
N_choices = family_id_choice_to_pref_cost.shape[1]
N_choices

11

In [8]:
family_id_choice_to_pref_cost

array([[   0,   50,   86, ...,  544, 1440, 2236],
       [   0,   50,   86, ...,  544, 1440, 2236],
       [   0,   50,   77, ...,  508, 1205, 1802],
       ...,
       [   0,   50,  104, ...,  616, 1910, 3104],
       [   0,   50,   95, ...,  580, 1675, 2670],
       [   0,   50,   86, ...,  544, 1440, 2236]])

## Ortools - CBC MIP solver

In [9]:
from ortools.linear_solver import pywraplp

In [10]:
solver = pywraplp.Solver('', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [11]:
print('Set num threads:', solver.SetNumThreads(num_cpu_cores))
print('Set time limit:', solver.SetTimeLimit(time_limit))

Set num threads: True
Set time limit: None


## Variables

In [12]:
# Variables
# assignment matrix[family, pref_rank]
assignment_matrix = {}
for family in families:
    for c in range(N_choices):
        assignment_matrix[family, c] = solver.BoolVar('x[%i,%i]' % (family, c))

In [13]:
len(assignment_matrix)

55000

In [14]:
possible_family_sizes = np.unique(n_people)

In [15]:
# unpreferred_day_counts[day, size]
unpreferred_day_counts = {}
for day in days:
    for size in possible_family_sizes:
        ub = int(N_max_people / size)
        # ub = solver.infinity()
        unpreferred_day_counts[day, size] = solver.IntVar(0, ub, 'd[%i,%i]' % (day, size))

In [16]:
len(unpreferred_day_counts)

700

In [17]:
# Occupancy matrix [day, N_d, N_d+1]
occupancy_matrix = {}
for day in days:
    if day < N_days:
        for o in allowed_occupancy:
            for o_next in allowed_occupancy:
                occupancy_matrix[day, o, o_next] = solver.BoolVar('o[%i,%i,%i]' % (day, o, o_next))
    else:
        for o in allowed_occupancy:
            occupancy_matrix[day, o] = solver.BoolVar('o[%i,%i]' % (day, o))

In [18]:
len(occupancy_matrix)

3066800

## Constraints

### constraint 1: each family only take one day (choice)

In [19]:
# Constraints
# constraint 1: each family only take one day (choice)
for family in families:
    solver.Add(solver.Sum([assignment_matrix[family, c]
                           for c in range(N_choices)]) == 1)

### constraint 2: each day can only have 125-300 people

In [20]:
# constraint 2: each day can only have 125-300 people

In [21]:
family_pref

array([[ 52,  38,  12, ...,  76,  10,  28],
       [ 26,   4,  82, ...,   6,  66,  61],
       [100,  54,  25, ...,  89,  80,  33],
       ...,
       [ 32,  66,  54, ...,  81,   3,   7],
       [ 67,  92,   4, ...,  12,  26,  70],
       [ 13,  11,  25, ...,  39,  18,  47]])

In [22]:
N_family_pref = N_choices - 1
N_family_pref

10

In [23]:
# day to dictionary of families who choice this day with value as preference rank
days_family_prefered = [{} for day in range(N_days+1)]  # day = 0 should not be used

In [24]:
for family, pref in enumerate(family_pref):
    for pref_rank, day in enumerate(pref):
        days_family_prefered[day][family] = pref_rank

In [25]:
# occupancy count [intermediate variables]
occupancy_counts = {}
for day in days:
    # find those family who like this day
    family_prefered = days_family_prefered[day]
    occupancy_counts[day] = (
        solver.Sum(
            [assignment_matrix[family, pref_rank] * n_people[family] 
             for family, pref_rank in family_prefered.items()]
        ) + 
        solver.Sum(
            [unpreferred_day_counts[day, size] * size for size in possible_family_sizes]
        )
    )

In [26]:
# for day in days:
#     # find those family who like this day
#     solver.Add(occupancy_counts[day] <= N_max_people, 'ub[%i]' % day)
#     solver.Add(occupancy_counts[day] >= N_min_people, 'ub[%i]' % day)

### constraint 3: unpreferred day family count conservation for each family size

In [27]:
# constraint 3: unpreferred day family count conservation for each family size

In [28]:
family_size_to_family_ids = {
    size: np.where(n_people == size)[0] for size in possible_family_sizes
}

In [29]:
for size in possible_family_sizes:
    solver.Add(
        solver.Sum([assignment_matrix[family, N_choices - 1]
                    for family in family_size_to_family_ids[size]])
        == solver.Sum([unpreferred_day_counts[day, size] for day in days]),
        'unpreferred_day_counts[%i]' % size
    )

### constrain 4: link occupancy boolean matrix to occupancy count

In [30]:
for day in days:
    if day < N_days:
        sum_from_occupancy_matrix = solver.Sum(
            [occupancy_matrix[day, o, o_next] * o 
             for o in allowed_occupancy for o_next in allowed_occupancy]
        )
    else:
        sum_from_occupancy_matrix = solver.Sum(
            [occupancy_matrix[day, o] * o for o in allowed_occupancy]
        )
    solver.Add(occupancy_counts[day] == sum_from_occupancy_matrix)

In [31]:
# occupancy boolean matrix normalization
# each day only take 1 occupancy value
for day in days:
    if day < N_days:
        occupancy_normalization = solver.Sum([
            occupancy_matrix[day, o, o_next]
            for o in allowed_occupancy for o_next in allowed_occupancy
        ])
    else:
        occupancy_normalization = solver.Sum([
            occupancy_matrix[day, o]
            for o in allowed_occupancy
        ])
    solver.Add(occupancy_normalization == 1)

In [32]:
# next day occupancy consistency
# Approach 1:
for day in days:
    if day < N_days:
        sum_from_next_occupancy_matrix = solver.Sum(
            [occupancy_matrix[day, o, o_next] * o_next 
             for o in allowed_occupancy for o_next in allowed_occupancy]
        )
        solver.Add(occupancy_counts[day + 1] == sum_from_next_occupancy_matrix)
# Approach 2:
# for day in days:
#     if day + 1 < N_days:
#         for o in allowed_occupancy:
#             solver.Add(
#                 solver.Sum(
#                     [occupancy_matrix[day, o_other, o] for o_other in allowed_occupancy]
#                 ) == solver.Sum(
#                     [occupancy_matrix[day + 1, o, o_other] for o_other in allowed_occupancy]
#                 )
#             )
# for o in allowed_occupancy:
#     solver.Add(
#         solver.Sum(
#             [occupancy_matrix[N_days - 1, o_other, o] for o_other in allowed_occupancy]
#         ) == occupancy_matrix[N_days, o]
#     )

## Objective

In [33]:
# Objective - Preference cost only as approximation
solver.Minimize(
    solver.Sum([
        assignment_matrix[family, c] * family_id_choice_to_pref_cost[family, c]
        for family in families for c in range(N_choices)
    ]) +
    solver.Sum([
        occupancy_matrix[day, o, o_next] * nd_ndp1_to_account_penality[o, o_next]
        for o in allowed_occupancy for o_next in allowed_occupancy
        for day in days if day < N_days
    ]) +
    solver.Sum([
        occupancy_matrix[N_days, o] * nd_ndp1_to_account_penality[o, o]
        for o in allowed_occupancy
    ])
)

## Solve

In [34]:
len(solver.variables())

3122500

In [35]:
%%time
# Solve
sol = solver.Solve()

resdict = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
           4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}
print('Result: ', resdict[sol])
print('Total cost = ', solver.Objective().Value())
print("Time = ", solver.WallTime(), " milliseconds")

Result:  NOT_SOLVED
Total cost =  0.0
Time =  21964054  milliseconds
CPU times: user 11h 42min 6s, sys: 2h 48min 49s, total: 14h 30min 56s
Wall time: 6h 4min 23s


In [36]:
# 20: 45338.0 OPTIMAL
# 40: 45338.0 OPTIMAL
# 60: 45338.0 FEASIBLE
#100: 45338.0 FEASIBLE
#10h (2h): 43999.0 FEASIBLE

## Solution

In [37]:
assignment_choices = np.array([
    [assignment_matrix[family, c].solution_value() for c in range(N_choices)]
    for family in families
]).argmax(axis=1)

In [38]:
assignment_choices

array([0, 0, 0, ..., 0, 0, 0])

In [39]:
assigned_day_new_raw = np.array([
    family_pref[family, c] if c < N_family_pref else -1 
    for family, c in enumerate(assignment_choices)
])

In [40]:
assigned_day_new_raw

array([ 52,  26, 100, ...,  32,  67,  13])

In [41]:
unpreferred_day_counts_sol = {
    size: [0]+[int(unpreferred_day_counts[day, size].solution_value()) for day in days]
    for size in possible_family_sizes
}

In [42]:
print('Unpreferred families slots:')
{size: sum(counts) for size, counts in unpreferred_day_counts_sol.items()}

Unpreferred families slots:


{2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}

In [43]:
def distribute_unpreferred_day(assigned_day, unpreferred_day_counts_sol, n_people):
    """ Distribute unpreferred day to each family who has -1 day assigned """
    assigned_day = assigned_day.copy()
    unpreferred_days = {size: [] for size in possible_family_sizes}
    for size in possible_family_sizes:
        for day, quota in enumerate(unpreferred_day_counts_sol[size]):
            unpreferred_days[size] = unpreferred_days[size] + [day] * quota
    unpreferred_day_headers = {size: 0 for size in possible_family_sizes}
    for family, (day, size) in enumerate(zip(assigned_day, n_people)):
        if day == -1:
            assigned_day[family] = unpreferred_days[size][unpreferred_day_headers[size]]
            unpreferred_day_headers[size] += 1
    return assigned_day

In [44]:
assigned_day_new = distribute_unpreferred_day(assigned_day_new_raw, unpreferred_day_counts_sol, n_people)

In [45]:
print('N family unpreferred assigned:', (~(assigned_day_new == assigned_day_new_raw)).sum())

N family unpreferred assigned: 0


In [46]:
def assigned_day_to_family_on_day(assigned_day):
    family_on_day = [set() for _ in range(N_days+1)] # 0 is empty set
    for i, day in enumerate(assigned_day):
        family_on_day[day].add(i)
    return family_on_day

def assigned_day_to_occupancy(assigned_day):
    occupancy = np.zeros(N_days+2, dtype='int32') # 0 is 0
    for i, n in enumerate(n_people):
        occupancy[assigned_day[i]] += n
    occupancy[0] = 125
    occupancy[-1] = occupancy[-2]
    return occupancy

In [47]:
family_on_day_new = assigned_day_to_family_on_day(assigned_day_new)
occupancy_new = assigned_day_to_occupancy(assigned_day_new)

In [50]:
try:
    is_valid = deep_check(assigned_day_new, family_on_day_new, occupancy_new)
except:
    is_valid = False
print('Valid solution:', is_valid)
print('Total score:    ', cal_total(assigned_day_new, occupancy_new))
print('Preference cost:', cal_total_preference(assigned_day_new))
print('Accounting cost:', cal_total_accounting(occupancy_new))

Total score:     1.0647073818721987e+90
Preference cost: 0
Accounting cost: 1.0647073818721987e+90


## Output

In [51]:
dump_conf(assigned_day_new, path_dump_improved)

## Debug

In [52]:
[
    [assignment_matrix[family, c].solution_value() for c in range(N_choices)]
    for family in range(10)
]        

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [53]:
[
    [unpreferred_day_counts[day, size].solution_value() for size in possible_family_sizes]
    for day in range(1, 10)
]        

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]